In [1]:
######################## PyTorch Implementation of Linear Regression with a Toy Problem #####################

# Import different packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
import random

print('PyTorch Version',torch.__version__)
print('Numpy Version',np.__version__,'\n')
################################################################
torch.manual_seed(0)   #You can use torch.manual_seed() to seed the RNG for all devices (both CPU and CUDA)
random.seed(0)         #For custom operators, you might need to set python seed
np.random.seed(0)      #If you or any of the libraries you are using rely on NumPy, you can seed the global NumPy RNG
################################################################

# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    best_loss = np.Inf
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Save best model based on loss
            if loss.detach().numpy() < best_loss:
                print('Loss improved from {:.2f} to {:.2f}'.format(best_loss,loss.detach().numpy()))
                best_loss = loss.detach().numpy()
                
                torch.save(model, 'best_linreg_model.pth')
            
            # 4. Compute gradients
            loss.backward()
            
            # 5. Update parameters using gradients
            opt.step()
            
            # 6. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) == num_epochs:
            #print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            print('\nBest Loss: {:.2f}\n'.format(best_loss))
            model = torch.load('best_linreg_model.pth')
            
            for key in model.state_dict():
                print('key: ', key)
                param = model.state_dict()[key]
                print(param)

                
######################################################################            
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets,'\n')


#########################################################################
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

# Define data loader
batch_size = 3  # should be << number of observations
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

for xb, yb in train_dl:
    print(xb)
    print(yb,'\n')
    break

    
###########################################################################
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias,'\n')

# Define loss function
loss_fn = F.mse_loss

# SGD Optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

fit(100, model, loss_fn, opt, train_dl)

PyTorch Version 1.5.1+cu101
Numpy Version 1.18.5 

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]) 

tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [ 56.,  70.]]) 

Parameter containing:
tensor([[-0.0114,  0.4578, -0.0512],
        [ 0.1528, -0.1745, -0.1135]], requires_grad=True)
Parameter containing:
tensor([-0.5516, -0.3824], requires_grad=True) 

Loss improved from inf to 7117.82
Loss improved from 7117.82 to 4780.08
Loss improved from 4780.08 to 3116.22
Loss improved from 3116.22 to 2983.52
Loss improved from 2983.52 to 1676.73
Loss improved from 1676.73 to 1481.72
Loss improved from 1481.72 to 802.22
Loss improved from 802.22 to 733.97
Loss improved from 733.97 to 542.07
Loss improved from 5

In [2]:
# Generate predictions
model = torch.load('best_linreg_model.pth')
model.eval()

preds = model(inputs)
preds

tensor([[ 58.8338,  72.8634],
        [ 79.4949,  97.9324],
        [120.2446, 130.5661],
        [ 32.4629,  55.3621],
        [ 90.5918, 103.9203]], grad_fn=<AddmmBackward>)

In [3]:
# Actual outputs
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [4]:
# Inferencing a given input
model(torch.tensor([[75, 63, 44.]]))  #testing

tensor([[55.1752, 70.3553]], grad_fn=<AddmmBackward>)

In [5]:
# Verify the best model's weights and bias
for key in model.state_dict():
    print('key: ', key)
    param = model.state_dict()[key]
    print(param)

key:  weight
tensor([[-0.1250,  0.8999,  0.1911],
        [ 0.0995,  0.7631,  0.3453]])
key:  bias
tensor([-0.5509, -0.3800])
